In [1]:
from Mollification import *

from ex_lscm import *

import trimesh
import os
import time

import meshio as mio

from buildGlueMap import *
from getPatch import *

root_folder = os.getcwd()

In [2]:
# read paths from a text file, each line is a path
paths = []
# make sure encoding = "utf-16 le" for the tooNiceMeshPaths.txt file


# with open("filtering-thing10k/veryBadMeshPaths.txt", "r", encoding="utf-16") as f:
#     for line in f:
#         paths.append(line.strip())

with open("filtering-thing10k/x2.txt", "r", encoding="utf-16") as f:
    for line in f:
        paths.append(line.strip())

# with open("filtering-thing10k/validMeshPaths.txt", "r", encoding="utf-16") as f:
#     for line in f:
#         paths.append(line.strip())

# with open("filtering-thing10k/tooNiceMeshPaths.txt", "r", encoding="utf-16") as f:
#     for line in f:
#         paths.append(line.strip())

# with open("filtering-thing10k/large_tooNiceMeshPaths.txt", "r", encoding="utf-16") as f:
#     for line in f:
#         paths.append(line.strip())
# with open("filtering-thing10k/large_kindaBadMeshPaths.txt", "r", encoding="utf-16") as f:
#     for line in f:
#         paths.append(line.strip())

# randomly order paths
import random
random.seed(41)
random.shuffle(paths)


print("Number of paths: ", len(paths))


Number of paths:  3706


In [3]:
# filter out paths to be only paths that have boundary edges
new_paths = []
for i in range(len(paths)):
    try:
        mesh = mio.read("../../GC/" + paths[i])
        F = mesh.cells_dict["triangle"]
        V = mesh.points
    except:
        print("Error reading mesh: ", i, paths[i])
        continue

    if np.shape(F)[0] > 20000:
        continue

    if (i+1)%300 == 0:
        print("Processing mesh ", i+1, " out of ", len(paths), " Meshes: ", len(new_paths))

    # check if mesh has boundary
    # E = igl.boundary_facets(F)
    # if(len(E) > 0):
    new_paths.append(paths[i])

paths = new_paths

print("Number of paths with boundary: ", len(paths))
        

Error reading mesh:  107 Thingi10k\Thingi10k\raw_meshes\47774.stl
Processing mesh  300  out of  3706  Meshes:  298


In [4]:
# save paths to a text file
with open("filtering-thing10k/x2.txt", "w", encoding="utf-16") as f:
    for path in paths:
        f.write(path + "\n")

In [28]:
mesh = mio.read("../../GC/" + paths[9])
F = mesh.cells_dict["triangle"]
V = mesh.points

FL = IntrinsicMollificationConstant(V, F, 1e-5)[-1]

V_uv = lscm_L(V, F, FL)[-1]

#print("V_uv" , V_uv)

qc = quasi_conformal_error(V, F, V_uv)[0]

print("qc", qc)



qc inf


In [3]:
delta = [1e-4, 1e-2] #, 1e-3, 1e-2, 1e-1]

def func(FL, G, delta, i):
    if i == 0:
        return FL, 0, 0
    elif i == 1:
        return IntrinsicMollificationFL(FL, G, delta, scheme=MOLLIFICATION_SCHEME.CONSTANT_EPSILON)
    elif i == 2:
        return IntrinsicMollificationFL(FL, G, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.CONSTANT_EPSILON)
    elif i == 3:
        return IntrinsicMollificationFL(FL, G, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_STEP)
    elif i == 4:        
        return IntrinsicMollificationFL(FL, G, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_INTERPOLATED)
    elif i == 5:
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_MANHATTAN)
    elif i == 6:
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_EUCLIDEAN)
    elif i == 7:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.GLOBAL_OPTIMIZATION_MANHATTAN)
    elif i == 8:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.GLOBAL_OPTIMIZATION_EUCLIDEAN)
    elif i == 9:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_STEP, pooling=MOLLIFICATION_POOLING.MEAN)
    elif i == 10:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_INTERPOLATED, pooling=MOLLIFICATION_POOLING.MEAN)
    elif i == 11:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_MANHATTAN, pooling=MOLLIFICATION_POOLING.MEAN)
    elif i == 12:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_EUCLIDEAN, pooling=MOLLIFICATION_POOLING.MEAN)
    elif i == 13:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_STEP, pooling=MOLLIFICATION_POOLING.MAX)
    elif i == 14:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_INTERPOLATED, pooling=MOLLIFICATION_POOLING.MAX)
    elif i == 15:
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_MANHATTAN, pooling=MOLLIFICATION_POOLING.MAX)
    elif i == 16:
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_EUCLIDEAN, pooling=MOLLIFICATION_POOLING.MAX)


time_glue = []

moll_labels = [
    "No Mollification",
    "Constant Epsilon",

    "Local: Constant Epsilon",
    "Local: One by One Step",
    "Local: One by One Interpolated",
    "Local: Least Mollification Manhattan",
    "Local: Least Mollification Euclidean",

    "Global: Least Mollification Manhattan",
    "Global: Least Mollification Euclidean"

    # "One by One Step (Global, Mean)",
    # "One by One Interpolated (Global, Mean)",
    # "Local Least Mollification Manhattan (Global, Mean)",
    # "Local Least Mollification Euclidean (Global, Mean)",

    # "One by One Step (Global, Max)",
    # "One by One Interpolated (Global, Max)",
    # "Local Least Mollification Manhattan (Global, Max)",
    # "Local Least Mollification Euclidean (Global, Max)"
]


n_alg = len(moll_labels)
n_delta = len(delta)

times = np.empty((n_alg, n_delta), dtype=object)
n_moll = np.empty((n_alg, n_delta), dtype=object)
avg_moll = np.empty((n_alg, n_delta), dtype=object)
avg_moll_for_mollified_edges = np.empty((n_alg, n_delta), dtype=object)

avg_qc_lscm = np.empty((n_alg, n_delta), dtype=object)
avg_qc_scp = np.empty((n_alg, n_delta), dtype=object)
avg_qc_harmonic = np.empty((n_alg, n_delta), dtype=object)

avg_len_dist_lscm = np.empty((n_alg, n_delta), dtype=object)
avg_len_dist_scp = np.empty((n_alg, n_delta), dtype=object)
avg_len_dist_harmonic = np.empty((n_alg, n_delta), dtype=object)

avg_area_dist_lscm = np.empty((n_alg, n_delta), dtype=object)
avg_area_dist_scp = np.empty((n_alg, n_delta), dtype=object)
avg_area_dist_harmonic = np.empty((n_alg, n_delta), dtype=object)


for i in range(n_alg):
    for j in range(n_delta):
        times[i][j] = []
        n_moll[i][j] = []
        avg_moll[i][j] = []
        avg_moll_for_mollified_edges[i][j] = []
        avg_qc_lscm[i][j] = []
        avg_qc_scp[i][j] = []
        avg_qc_harmonic[i][j] = []
        avg_len_dist_lscm[i][j] = []
        avg_len_dist_scp[i][j] = []
        avg_len_dist_harmonic[i][j] = []
        avg_area_dist_lscm[i][j] = []
        avg_area_dist_scp[i][j] = []
        avg_area_dist_harmonic[i][j] = []

n_qc_nan_lscm = np.zeros((n_alg, n_delta))
n_qc_nan_scp = np.zeros((n_alg, n_delta))
n_qc_nan_harmonic = np.zeros((n_alg, n_delta))
n_len_dist_nan_lscm = np.zeros((n_alg, n_delta))
n_len_dist_nan_scp = np.zeros((n_alg, n_delta))
n_len_dist_nan_harmonic = np.zeros((n_alg, n_delta))
n_area_dist_nan_lscm = np.zeros((n_alg, n_delta))
n_area_dist_nan_scp = np.zeros((n_alg, n_delta))
n_area_dist_nan_harmonic = np.zeros((n_alg, n_delta))


n = 0

for i in range(len(paths)):
    try:

        mesh = mio.read("../../GC/" + paths[i])
        F = mesh.cells_dict["triangle"]
        V = mesh.points

        # [V,F]= igl.read_triangle_mesh(os.path.join(
                    # root_folder, "../../GC/", paths[i]))
        print("Mesh ", i + 1, " out of ", len(paths), ". nF: ", len(F), ". nV: ", len(V), ", eul = " , igl.euler_characteristic(F), "loop count: ", len(igl.all_boundary_loop(F)))

        # check if mesh has boundary
        # E = igl.boundary_facets(F)
        # if(len(E) == 0):
        #     continue

        start = time.time()
        G = build_gluing_map(F)
        end = time.time()
        time_glue.append(end - start)


        # Better? : use the glueing map and BFS to get a patch of faces
        # and use that as the mesh for parameterization instead of the whole mesh
        factor = 0.2

        F_Patch = get_patch(F, G, factor)
        # remove unused vertices
        V_Patch, F_Patch = igl.remove_unreferenced(V, F_Patch)[0:2]
        assert len(V_Patch) == np.max(F_Patch) + 1
        #print("1 xai = ", igl.euler_characteristic(F_Patch), "loop count: ", len(igl.all_boundary_loop(F_Patch)) , "mesh: " , i + 1, " path: ", paths[i])

        ignore = False
        # TODO: check for vertex manifoldness aswell.
        while igl.euler_characteristic(F_Patch) + len(igl.all_boundary_loop(F_Patch)) != 2 or igl.is_edge_manifold(F_Patch) == False:
            factor *= 0.98
            #print("2 xai = ", igl.euler_characteristic(F_Patch), "loop count: ", len(igl.all_boundary_loop(F_Patch)) , "mesh: " , i + 1, " path: ", paths[i])
            F_Patch = get_patch(F, G, factor)
            # remove unused vertices
            V_Patch, F_Patch = igl.remove_unreferenced(V, F_Patch)[0:2]
            assert len(V_Patch) == np.max(F_Patch) + 1

            if factor < 0.15:
                ignore = True
                break

        if ignore:
            continue

        F = F_Patch
        V = V_Patch

        print("5 xai = ", igl.euler_characteristic(F), "loop count: ", len(igl.all_boundary_loop(F)) , "mesh: " , i + 1, " path: ", paths[i], "nF: ", len(F), "nV: ", len(V))

        # recalculate the gluing map for the new mesh (patch)
        G = build_gluing_map(F)

        FL0 = igl.edge_lengths(V, F)

        avg_edge_length = np.mean(FL0)

    except:
        print("Error reading, gluing, or getting patch for mesh: ", i + 1, " path: ", paths[i])
        continue

    # ps.init()

    # # register mesh with polyscope
    # ps_mesh = ps.register_surface_mesh("mesh", V, F, edge_width=1)

    # ps.show()

    # after polyscope is closed, continue

    for j in range(n_delta):
        for k in range(n_alg):
            # if k > 7:
            #     continue
            try:
                start = time.time()
                FL, nMoll, nIter = func(FL0, G, delta[j], k)
                end = time.time()
                times[k][j].append(end - start)
                n_moll[k][j].append(nMoll)

                try:
                    V_uv = lscm_L(V, F, FL)[-1]
                    qc = quasi_conformal_error(V, F, V_uv)[0]
                    len_dist = isometric_distortion(V, F, V_uv)
                    area_dist = area_distortion(V, F, V_uv)
                    #print("lqc = ", qc)
                    if np.isnan(qc) or np.isinf(qc):
                        n_qc_nan_lscm[k][j] += 1
                        qc = np.inf
                    
                    if np.isnan(len_dist) or np.isinf(len_dist):
                        n_len_dist_nan_lscm[k][j] += 1
                        len_dist = np.inf

                    if np.isnan(area_dist) or np.isinf(area_dist):
                        n_area_dist_nan_lscm[k][j] += 1
                        area_dist = np.inf

                    avg_qc_lscm[k][j].append(qc)
                    avg_len_dist_lscm[k][j].append(len_dist)
                    avg_area_dist_lscm[k][j].append(area_dist)

                except:
                    avg_qc_lscm[k][j].append(np.inf)
                    avg_len_dist_lscm[k][j].append(np.inf)
                    avg_area_dist_lscm[k][j].append(np.inf)
                    n_qc_nan_lscm[k][j] += 1
                    n_len_dist_nan_lscm[k][j] += 1
                    n_area_dist_nan_lscm[k][j] += 1

                if k != 0 and qc < 1.4 and avg_qc_lscm[0][j][-1] > 2.5 and avg_qc_lscm[0][j][-1] < 10:
                    # save this patch into a file with the same name as the mesh file
                    # save_path: "../output/lscm_example/" + paths[i] - Thingi10k\Thingi10k\raw_meshes\
                    print("saving lscm example")
                    save_path = "../output/lscm_example/" + paths[i][len("Thingi10k\\Thingi10k\\raw_meshes\\"):]
                    # save:
                    igl.write_triangle_mesh(save_path, V, F)

                try:
                    V_uv = scp_L(F, FL)[-1]
                    qc = quasi_conformal_error(V, F, V_uv)[0]
                    len_dist = isometric_distortion(V, F, V_uv)
                    area_dist = area_distortion(V, F, V_uv)
                    #print("sqc = ", qc)
                    if np.isnan(qc) or np.isinf(qc):
                        n_qc_nan_scp[k][j] += 1
                        qc = np.inf
                    
                    if np.isnan(len_dist) or np.isinf(len_dist):
                        n_len_dist_nan_scp[k][j] += 1
                        len_dist = np.inf

                    if np.isnan(area_dist) or np.isinf(area_dist):
                        n_area_dist_nan_scp[k][j] += 1
                        area_dist = np.inf

                    avg_qc_scp[k][j].append(qc)
                    avg_len_dist_scp[k][j].append(len_dist)
                    avg_area_dist_scp[k][j].append(area_dist)



                except:
                    avg_qc_scp[k][j].append(np.inf)
                    avg_len_dist_scp[k][j].append(np.inf)
                    avg_area_dist_scp[k][j].append(np.inf)
                    n_qc_nan_scp[k][j] += 1
                    n_len_dist_nan_scp[k][j] += 1
                    n_area_dist_nan_scp[k][j] += 1

                if k != 0 and qc < 1.4 and avg_qc_scp[0][j][-1] > 2.5 and avg_qc_scp[0][j][-1] < 10:
                    # save this patch into a file with the same name as the mesh file
                    # save_path: "../output/lscm_example/" + paths[i] - Thingi10k\Thingi10k\raw_meshes\
                    print("saving scp example")
                    save_path = "../output/scp_example/" + paths[i][len("Thingi10k\\Thingi10k\\raw_meshes\\"):]
                    # save:
                    igl.write_triangle_mesh(save_path, V, F)

                try:
                    V_uv = harmonic_L(V, F, FL)
                    qc = quasi_conformal_error(V, F, V_uv)[0]
                    len_dist = isometric_distortion(V, F, V_uv)
                    area_dist = area_distortion(V, F, V_uv)
                    #print("hqc = ", qc)
                    if np.isnan(qc) or np.isinf(qc):
                        n_qc_nan_harmonic[k][j] += 1
                        qc = np.inf

                    if np.isnan(len_dist) or np.isinf(len_dist):
                        n_len_dist_nan_harmonic[k][j] += 1
                        len_dist = np.inf

                    if np.isnan(area_dist) or np.isinf(area_dist):
                        n_area_dist_nan_harmonic[k][j] += 1
                        area_dist = np.inf

                    avg_qc_harmonic[k][j].append(qc)
                    avg_len_dist_harmonic[k][j].append(len_dist)
                    avg_area_dist_harmonic[k][j].append(area_dist)
                
                except:
                    avg_qc_harmonic[k][j].append(np.inf)
                    avg_len_dist_harmonic[k][j].append(np.inf)
                    avg_area_dist_harmonic[k][j].append(np.inf)
                    n_qc_nan_harmonic[k][j] += 1
                    n_len_dist_nan_harmonic[k][j] += 1
                    n_area_dist_nan_harmonic[k][j] += 1

                if k != 0 and ((len_dist < 1 and avg_len_dist_harmonic[0][j][-1] > 2.5 and avg_len_dist_harmonic[0][j][-1] < 10)   \
                              or (area_dist < 1 and avg_area_dist_harmonic[0][j][-1] > 2.5 and avg_area_dist_harmonic[0][j][-1] < 10)):
                    # save this patch into a file with the same name as the mesh file
                    # save_path: "../output/lscm_example/" + paths[i] - Thingi10k\Thingi10k\raw_meshes\
                    print("saving harmonic example")
                    save_path = "../output/harmonic_example/" + paths[i][len("Thingi10k\\Thingi10k\\raw_meshes\\"):]
                    # save:
                    igl.write_triangle_mesh(save_path, V, F)

                avg_moll[k][j].append(np.sum(FL - FL0) / np.sum(FL0))
                if(nMoll > 0):
                    avg_moll_for_mollified_edges[k][j].append(np.sum(FL - FL0) / (avg_edge_length * nMoll * 3))
            except:
                print("Error in mesh ", i + 1, " path: ", paths[i], " delta: ", delta[j], " mollification: ", moll_labels[k])
                continue
    
    print(i + 1)

    n += 1
    # except:
    #     print("Error reading mesh: ", i, paths[i])
    #     continue

    if((i + 1) % 10 == 0):
        print("Finished ", i + 1, " meshes.")
        print("Avg time to glue: ", np.mean(time_glue), " Std dev time to glue: ", np.std(time_glue))
        print("_________________________________________________________")

        for j in range(n_delta):
            for k in range(n_alg):
                # if k > 7:
                #     continue
                print("Delta: ", delta[j], " Mollification: ", moll_labels[k])
                print("Avg time: ", np.sum(times[k][j]) / (n))
                print("Avg number of mollified edges: ", np.sum(n_moll[k][j]) / (n))
                print("Avg mollification: ", np.sum(avg_moll[k][j]) / n)
                print("Avg mollification for mollified edges: ", np.sum(avg_moll_for_mollified_edges[k][j]) / n)
                print("LSCM, QC: # nan/inf: ", n_qc_nan_lscm[k][j] , "Median: ", np.median(avg_qc_lscm[k][j]))
                print("LSCM, len dist: # nan/inf: ", n_len_dist_nan_lscm[k][j] , "Median: ", np.median(avg_len_dist_lscm[k][j]))
                print("LSCM, area dist: # nan/inf: ", n_area_dist_nan_lscm[k][j] , "Median: ", np.median(avg_area_dist_lscm[k][j]))
                print("SCP, QC: # nan/inf: ", n_qc_nan_scp[k][j] , "Median: ", np.median(avg_qc_scp[k][j]))
                print("SCP, len dist: # nan/inf: ", n_len_dist_nan_scp[k][j] , "Median: ", np.median(avg_len_dist_scp[k][j]))
                print("SCP, area dist: # nan/inf: ", n_area_dist_nan_scp[k][j] , "Median: ", np.median(avg_area_dist_scp[k][j]))
                print("H, QC: # nan/inf: ", n_qc_nan_harmonic[k][j] , "Median: ", np.median(avg_qc_harmonic[k][j]))
                print("H, len dist: # nan/inf: ", n_len_dist_nan_harmonic[k][j] , "Median: ", np.median(avg_len_dist_harmonic[k][j]))
                print("H, area dist: # nan/inf: ", n_area_dist_nan_harmonic[k][j] , "Median: ", np.median(avg_area_dist_harmonic[k][j]))
                
                # print("Median quasi conformal error(scp): ", np.median(avg_qc_scp[k][j]))
                # print("number of nan / inf qc(scp): ", n_qc_nan_scp[k][j])
                # print("Median quasi conformal error(harmonic): ", np.median(avg_qc_harmonic[k][j]))
                # print("number of nan / inf qc(harmonic): ", n_qc_nan_harmonic[k][j])
                print("_________________________________________________________")


print("Finished All meshes.")
print("Avg time to glue: ", np.mean(time_glue), " Std dev time to glue: ", np.std(time_glue))
print("_________________________________________________________")
for j in range(n_delta):
    for k in range(n_alg):
        print("Delta: ", delta[j], " Mollification: ", moll_labels[k])
        print("Avg time: ", times[k][j] / n)
        print("Avg number of mollified edges: ", n_moll[k][j] / n)
        print("Avg mollification: ", avg_moll[k][j] / n)
        print("Avg mollification for mollified edges: ", avg_moll_for_mollified_edges[k][j] / n)
        print("Median quasi conformal error(lscm): ", np.median(avg_qc_lscm[k][j]))
        print("number of nan / inf qc(lscm): ", n_qc_nan_lscm[k][j])
        print("Median quasi conformal error(scp): ", np.median(avg_qc_scp[k][j]))
        print("number of nan / inf qc(scp): ", n_qc_nan_scp[k][j])
        print("Median quasi conformal error(harmonic): ", np.median(avg_qc_harmonic[k][j]))
        print("number of nan / inf qc(harmonic): ", n_qc_nan_harmonic[k][j])
        print("_________________________________________________________")

Mesh  1  out of  3706 . nF:  1896 . nV:  950 , eul =  2 loop count:  0
5 xai =  1 loop count:  1 mesh:  1  path:  Thingi10k\Thingi10k\raw_meshes\72333.stl nF:  358 nV:  206


d:\4B\intrinsic_mollification\src\cotanLaplace.py:65: RuntimeWarning: divide by zero encountered in scalar divide
  wjk = (lengths[j] ** 2 + lengths[k] ** 2 - lengths[i] ** 2) / (8*area)
d:\4B\intrinsic_mollification\src\cotanLaplace.py:87: RuntimeWarning: invalid value encountered in scalar subtract
  Laplacian[fr[j], fr[j]] -= wjk  # L_jj
d:\4B\intrinsic_mollification\src\cotanLaplace.py:89: RuntimeWarning: invalid value encountered in scalar add
  Laplacian[fr[j], fr[k]] += wjk  # L_jk
d:\4B\intrinsic_mollification\src\cotanLaplace.py:90: RuntimeWarning: invalid value encountered in scalar add
  Laplacian[fr[k], fr[j]] += wjk  # L_kj
d:\4B\intrinsic_mollification\src\cotanLaplace.py:88: RuntimeWarning: invalid value encountered in scalar subtract
  Laplacian[fr[k], fr[k]] -= wjk  # L_kk
d:\4B\intrinsic_mollification\src\massmatrix.py:69: RuntimeWarning: invalid value encountered in sqrt
  return 0.25*np.sqrt((l[0]+l[1]+l[2])*


1
Mesh  2  out of  3706 . nF:  1076 . nV:  532 , eul =  -6 loop count:  0
5 xai =  1 loop count:  1 mesh:  2  path:  Thingi10k\Thingi10k\raw_meshes\162332.stl nF:  203 nV:  127
2
Mesh  3  out of  3706 . nF:  16166 . nV:  8085 , eul =  2 loop count:  0
5 xai =  1 loop count:  1 mesh:  3  path:  Thingi10k\Thingi10k\raw_meshes\100343.stl nF:  3234 nV:  1752
3
Mesh  4  out of  3706 . nF:  12342 . nV:  6173 , eul =  2 loop count:  0
5 xai =  1 loop count:  1 mesh:  4  path:  Thingi10k\Thingi10k\raw_meshes\94157.stl nF:  2469 nV:  1314
4
Mesh  5  out of  3706 . nF:  2632 . nV:  1310 , eul =  -6 loop count:  0
5 xai =  -5 loop count:  7 mesh:  5  path:  Thingi10k\Thingi10k\raw_meshes\95502.stl nF:  516 nV:  291
5
Mesh  6  out of  3706 . nF:  8878 . nV:  4441 , eul =  2 loop count:  0
5 xai =  1 loop count:  1 mesh:  6  path:  Thingi10k\Thingi10k\raw_meshes\66783.stl nF:  1776 nV:  970
6
Mesh  7  out of  3706 . nF:  1124 . nV:  560 , eul =  -2 loop count:  0
5 xai =  0 loop count:  2 mesh:  7 

d:\4B\intrinsic_mollification\src\cotanLaplace.py:65: RuntimeWarning: invalid value encountered in scalar divide
  wjk = (lengths[j] ** 2 + lengths[k] ** 2 - lengths[i] ** 2) / (8*area)
d:\4B\intrinsic_mollification\src\ex_lscm.py:236: RuntimeWarning: invalid value encountered in divide
  Ss = (q[0] * (p[1][1] - p[2][1]) + q[1] * (p[2][1] - p[0][1]) + q[2] * (p[0][1] - p[1][1])) / area_p
d:\4B\intrinsic_mollification\src\ex_lscm.py:237: RuntimeWarning: invalid value encountered in divide
  St = (q[0] * (p[2][0] - p[1][0]) + q[1] * (p[0][0] - p[2][0]) + q[2] * (p[1][0] - p[0][0])) / area_p
d:\4B\intrinsic_mollification\src\ex_lscm.py:237: RuntimeWarning: divide by zero encountered in divide
  St = (q[0] * (p[2][0] - p[1][0]) + q[1] * (p[0][0] - p[2][0]) + q[2] * (p[1][0] - p[0][0])) / area_p
d:\4B\intrinsic_mollification\src\ex_lscm.py:236: RuntimeWarning: divide by zero encountered in divide
  Ss = (q[0] * (p[1][1] - p[2][1]) + q[1] * (p[2][1] - p[0][1]) + q[2] * (p[0][1] - p[1][1])) /

16
Mesh  17  out of  3706 . nF:  1260 . nV:  632 , eul =  2 loop count:  0
5 xai =  1 loop count:  1 mesh:  17  path:  Thingi10k\Thingi10k\raw_meshes\133993.stl nF:  252 nV:  145
17
Mesh  18  out of  3706 . nF:  238 . nV:  121 , eul =  2 loop count:  0
5 xai =  1 loop count:  1 mesh:  18  path:  Thingi10k\Thingi10k\raw_meshes\1224383.stl nF:  48 nV:  32
18
Mesh  19  out of  3706 . nF:  920 . nV:  456 , eul =  -4 loop count:  0
5 xai =  1 loop count:  1 mesh:  19  path:  Thingi10k\Thingi10k\raw_meshes\87718.stl nF:  184 nV:  108
19
Mesh  20  out of  3706 . nF:  4172 . nV:  2088 , eul =  2 loop count:  0
5 xai =  1 loop count:  1 mesh:  20  path:  Thingi10k\Thingi10k\raw_meshes\72203.stl nF:  835 nV:  472
20
Finished  20  meshes.
Avg time to glue:  0.09225276708602906  Std dev time to glue:  0.11789167621618335
_________________________________________________________
Delta:  0.0001  Mollification:  No Mollification
Avg time:  0.0
Avg number of mollified edges:  0.0
Avg mollification:  0

: 

: 

In [ ]:
[V,F]= igl.read_triangle_mesh(os.path.join(
            # root_folder, "../../GC/TestEdgeLengths", "11.stl"))
            # root_folder, "../../GC/Thingi10K/Thingi10K/raw_meshes", "36069.stl"))
            # root_folder, "../../GC/Thingi10K/Thingi10K/raw_meshes", "93743.stl"))
            # root_folder, "../data", "goathead.obj"))
            root_folder, "../data", "dime_counting_tube.stl"))


In [ ]:
mesh = mio.read("../data/dime_counting_tube.stl")
F = mesh.cells_dict["triangle"]
V = mesh.points

6.0

In [ ]:
delta = 1e-2
FL = igl.edge_lengths(V, F)

In [ ]:
# time before
end0 = time.time()
FL1, nMoll1, _ = IntrinsicMollificationFL(FL, None, delta, scheme=MOLLIFICATION_SCHEME.CONSTANT_EPSILON)
end1 = time.time()
print("Constant Epsilon took: ", end1 - end0)

FL2, nMoll2, _ = IntrinsicMollificationFL(FL, None, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_STEP)
end2 = time.time()
print("One by one step took: ", end2 - end1)

FL3, nMoll3, _ = IntrinsicMollificationFL(FL, None, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_INTERPOLATED)
end3 = time.time()
print("One by one interpolated took: ", end3 - end2)

FL4, nMoll4, _ = IntrinsicMollificationFL(FL, None, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_MANHATTAN)
end4 = time.time()
print("Local least mollification Manhattan took: ", end4 - end3)

FL5, nMoll5, _ = IntrinsicMollificationFL(FL, None, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_EUCLIDEAN)
end5 = time.time()
print("Local least mollification Euclidean took: ", end5 - end4)

[0 1 0 0]
[0 2 0 2] >
[0 2 0 2]
[1 2 0 1] >
[1 2 0 1]
[3 4 1 0] >
[3 4 1 0]
[3 5 1 2] >
[3 5 1 2]
[4 5 1 1] >
[4 5 1 1]
[6 7 2 0] >
[6 7 2 0]
[6 8 2 2] >
[6 8 2 2]
[7 8 2 1] >
[7 8 2 1]
[ 9 10  3  0] >
[ 9 10  3  0]
[ 9 11  3  2] >
[ 9 11  3  2]
[10 11  3  1] >
[10 11  3  1]
[12 13  4  0] >
[12 13  4  0]
[12 14  4  2] >
[12 14  4  2]
[13 14  4  1] >
[13 14  4  1]
[15 16  5  0] >
[15 16  5  0]
[15 17  5  2] >
[15 17  5  2]
[16 17  5  1] >
[16 17  5  1]
[18 19  6  0] >
[18 19  6  0]
[18 20  6  2] >
[18 20  6  2]
[19 20  6  1] >
[19 20  6  1]
[21 22  7  0] >
[21 22  7  0]
[21 23  7  2] >
[21 23  7  2]
[22 23  7  1] >
[22 23  7  1]
[24 25  8  0] >
[24 25  8  0]
[24 26  8  2] >
[24 26  8  2]
[25 26  8  1] >
[25 26  8  1]
[27 28  9  0] >
[27 28  9  0]
[27 29  9  2] >
[27 29  9  2]
[28 29  9  1] >
[28 29  9  1]
[30 31 10  0] >
[30 31 10  0]
[30 32 10  2] >
[30 32 10  2]
[31 32 10  1] >
[31 32 10  1]
[33 34 11  0] >
[33 34 11  0]
[33 35 11  2] >
[33 35 11  2]
[34 35 11  1] >
[34 35 11  1]
[36 

IndexError: index 480 is out of bounds for axis 0 with size 480

In [ ]:
print(G)

In [ ]:
G = build_gluing_map(F)


end5 = time.time()

FL6, nMoll6, nIter6 = IntrinsicMollificationFL(L, G, delta, scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_STEP, pooling=MOLLIFICATION_POOLING.MEAN)
end6 = time.time()
print("Sequential global, one by one step, mean pooling took: ", end6 - end5)

 FL7, nMoll7, nIter7 = IntrinsicMollificationFL(L, G, delta, scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_INTERPOLATED, pooling=MOLLIFICATION_POOLING.MEAN)
 end7 = time.time()
 print("Sequential global, one by one interpolated, mean pooling took: ", end7 - end6)
 FL8, nMoll8, nIter8 = IntrinsicMollificationFL(L, G, delta, scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_MANHATTAN, pooling=MOLLIFICATION_POOLING.MEAN)
 end8 = time.time()
 print("Sequential global, local least mollification Manhattan, mean pooling took: ", end8 - end7)
 FL9, nMoll9, nIter9 = IntrinsicMollificationFL(L, G, delta, scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_EUCLIDEAN, pooling=MOLLIFICATION_POOLING.MEAN)
 end9 = time.time()
 print("Sequential global, local least mollification Euclidean, mean pooling took: ", end9 - end8)

1 27 27
2 108 135
3 33 168
4 36 204
5 36 240
6 35 275
7 35 310
8 35 345
9 35 380
10 35 415
11 35 450
12 35 485
13 35 520
14 35 555
15 35 590
16 35 625
17 35 660
18 34 694
19 34 728
20 34 762
21 33 795
22 33 828
23 33 861
24 30 891
25 25 916
26 22 938
27 11 949
28 6 955
29 3 958
30 1 959
31 0 959
Sequential global, one by one step, mean pooling took:  0.058502912521362305


In [ ]:
print("Constant Epsilon nMoll", nMoll1)
print("One by one step nMoll", nMoll2)
print("One by one interpolated nMoll", nMoll3)
print("Local least mollification Manhattan", nMoll4)
print("Local least mollification Euclidean", nMoll5) 
print("Sequential global, one by one step, mean pooling", nMoll6, "with", nIter6, "iterations")
print("Sequential global, one by one interpolated, mean pooling", nMoll7, "with", nIter7, "iterations")
print("Sequential global, local least mollification Manhattan, mean pooling", nMoll8, "with", nIter8, "iterations")
print("Sequential global, local least mollification Euclidean, mean pooling", nMoll9, "with", nIter9, "iterations")

Constant Epsilon nMoll 699466
One by one step nMoll 621
One by one interpolated nMoll 621
Local least mollification Manhattan 621
Local least mollification Euclidean 621
Sequential global, one by one step, mean pooling 621 with 2 iterations
Sequential global, one by one interpolated, mean pooling 621 with 2 iterations
Sequential global, local least mollification Manhattan, mean pooling 621 with 2 iterations
Sequential global, local least mollification Euclidean, mean pooling 621 with 2 iterations


In [ ]:
print(np.sum(FL1 - FL0) / np.sum(FL0))
print(np.sum(FL2 - FL0) / np.sum(FL0))
print(np.sum(FL3 - FL0) / np.sum(FL0))
print(np.sum(FL4 - FL0) / np.sum(FL0))
print(np.sum(FL5 - FL0) / np.sum(FL0))
print(np.sum(FL6 - FL0) / np.sum(FL0))
print(np.sum(FL7 - FL0) / np.sum(FL0))
print(np.sum(FL8 - FL0) / np.sum(FL0))
print(np.sum(FL9 - FL0) / np.sum(FL0))

0.009999969844763803
1.9515460139100948e-06
3.474920234048739e-06
1.9443048566123697e-06
1.9480259108943147e-06
1.9515460139100948e-06
3.474920234048739e-06
1.9443048566123697e-06
1.9480259108943147e-06


In [ ]:
F = np.array([F]).repeat(2, axis=0)
print(F.shape)

(2, 144046, 3)


In [ ]:
b = np.array([- delta,   - delta,    - delta,    0,     0,    0])
print(b, b[3])

[-0.1 -0.1 -0.1  0.   0.   0. ] 0.0


In [ ]:

print(times)
print(times[0][0])

[[[0.008999824523925781, 0.009971857070922852, 0.0074045658111572266, 0.007978677749633789, 0.007978200912475586, 0.007978677749633789, 0.006981372833251953, 0.0069806575775146484, 0.007978200912475586, 0.00698399543762207, 0.0020787715911865234, 0.01875782012939453, 0.012966156005859375, 0.20042777061462402, 0.21538996696472168, 0.22843718528747559, 0.24833154678344727, 0.22535395622253418, 0.23342514038085938, 0.2054445743560791, 0.22211956977844238, 0.22598624229431152, 0.21291351318359375, 0.22838973999023438, 0.2183852195739746, 0.2104785442352295, 0.0209505558013916, 0.0209810733795166, 0.02093958854675293, 0.02194070816040039, 0.02194833755493164, 0.020937442779541016, 0.02293872833251953, 0.024903535842895508, 0.02397322654724121, 0.023938894271850586, 0.020876646041870117, 0.025930166244506836, 0.025974512100219727, 0.042885780334472656, 0.04487967491149902, 0.044912099838256836, 0.04484891891479492, 0.05186605453491211, 0.04487323760986328, 0.040891408920288086, 0.04391884803

In [10]:
for j in range(len(delta)):
    for k in range(len(moll_labels)):
        if k > 7:
            continue
        print("Delta: ", delta[j], " Mollification: ", moll_labels[k])
        print("Avg time: ", times[k][j] / n)
        print("Avg number of mollified edges: ", n_moll[k][j] / n)
        print("Avg number of iterations: ", n_iter[k][j] / n)
        print("Avg mollification: ", avg_moll[k][j] / n)
        print("Avg mollification for mollified edges: ", avg_moll_for_mollified_edges[k][j] / n)
        print("Avg quasi conformal error(lscm): ", avg_qc_lscm[k][j] / n)
        print("number of nan / inf qc(lscm): ", n_qc_nan_lscm[k][j])
        print("Avg quasi conformal error(scp): ", avg_qc_scp[k][j] / n)
        print("number of nan / inf qc(scp): ", n_qc_nan_scp[k][j])
        print("_________________________________________________________")

Delta:  0.0001  Mollification:  No Mollification
Average time:  0.0
Average number of mollified edges:  0.0
Average number of iterations:  0.0
Average mollification:  0.0
Average mollification for mollified edges:  0.0
Average quasi conformal error(lscm):  7.07306951821784
number of nan / inf qc(lscm):  0.0
Average quasi conformal error(scp):  3.118959927356654
number of nan / inf qc(scp):  0.0
_________________________________________________________
Delta:  0.0001  Mollification:  Constant Epsilon
Average time:  0.0
Average number of mollified edges:  0.0
Average number of iterations:  1.0
Average mollification:  0.0
Average mollification for mollified edges:  0.0
Average quasi conformal error(lscm):  6.096769218962439
number of nan / inf qc(lscm):  0.0
Average quasi conformal error(scp):  8.448448098203851
number of nan / inf qc(scp):  0.0
_________________________________________________________
Delta:  0.0001  Mollification:  One by One Step (Local)
Average time:  0.00049889087677